In [1]:
from google.cloud import bigquery
import pandas as pd
import sqlite3

Connect to BigQuery and fetch the necessary data:

In [25]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# List of tables from umt-msba project (excluding inactive ones)
table_names = [
    'transArchive_201001_201003', 'transArchive_201004_201006', 'transArchive_201007_201009', 
    'transArchive_201010_201012', 'transArchive_201101_201103', 'transArchive_201104', 
    'transArchive_201105', 'transArchive_201106', 'transArchive_201107_201109', 
    'transArchive_201110_201112', 'transArchive_201201_201203', 'transArchive_201204_201206', 
    'transArchive_201207_201209', 'transArchive_201210_201212', 'transArchive_201301_201303', 
    'transArchive_201304_201306', 'transArchive_201307_201309', 'transArchive_201310_201312', 
    'transArchive_201401_201403', 'transArchive_201404_201406', 'transArchive_201407_201409', 
    'transArchive_201410_201412', 'transArchive_201501_201503', 'transArchive_201504_201506', 
    'transArchive_201507_201509', 'transArchive_201510', 'transArchive_201511', 'transArchive_201512', 
    'transArchive_201601', 'transArchive_201602', 'transArchive_201603', 'transArchive_201604', 
    'transArchive_201605', 'transArchive_201606', 'transArchive_201607', 'transArchive_201608', 
    'transArchive_201609', 'transArchive_201610', 'transArchive_201611', 'transArchive_201612', 
    'transArchive_201701'
]

# Construct a UNION query across all tables to gather data excluding non-owners (card_no != 3)
union_query = " UNION ALL ".join([
    f"SELECT datetime, card_no, department, description, upc, unitPrice, total, trans_status, "
    f"EXTRACT(YEAR FROM datetime) AS year, EXTRACT(MONTH FROM datetime) AS month, "
    f"EXTRACT(DAY FROM datetime) AS day, EXTRACT(HOUR FROM datetime) AS hour "
    f"FROM `umt-msba.transactions.{table_name}` WHERE card_no != 3" 
    for table_name in table_names
])

# Full query to fetch all owner transactions
full_query = f"""
    SELECT * FROM ({union_query})
"""

# Execute the query and fetch the data
df = client.query(full_query).to_dataframe()

# Display the DataFrame
print(df.head())


C:\Users\Riley_26\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


KeyboardInterrupt: 

In [10]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# Step 1: Fetch data from BigQuery to build the summary tables
query = """
    SELECT datetime, card_no, department, description, upc, unitPrice, total, trans_status, 
           EXTRACT(YEAR FROM datetime) AS year, EXTRACT(MONTH FROM datetime) AS month, 
           EXTRACT(DAY FROM datetime) AS day, EXTRACT(HOUR FROM datetime) AS hour
    FROM `umt-msba.transactions.transArchive_201001_201003`
    WHERE card_no != 3  -- Exclude non-owners
"""

# Execute the query and fetch the data
df = client.query(query).to_dataframe()

# Display the DataFrame
print(df.head())


C:\Users\Riley_26\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                   datetime  card_no  department description upc  unitPrice  \
0 2010-02-01 15:24:21+00:00  14140.0         0.0      Change   0        0.0   
1 2010-02-02 10:26:27+00:00  24359.0         0.0        Cash   0        0.0   
2 2010-03-02 20:14:41+00:00  15290.0         0.0       Check   0        0.0   
3 2010-03-03 12:56:55+00:00  20730.0         0.0      Change   0        0.0   
4 2010-03-05 13:10:30+00:00  14472.0         0.0      Change   0        0.0   

   total trans_status  year  month  day  hour  
0  11.00         None  2010      2    1    15  
1 -22.00         None  2010      2    2    10  
2 -15.01         None  2010      3    2    20  
3   0.80         None  2010      3    3    12  
4   7.00         None  2010      3    5    13  


Build Summary Tables  
    
      
      
      
1. Sales by date by hour:

In [17]:
# Create the 'Sales by date by hour' summary table
sales_by_hour = df.groupby(['year', 'month', 'day', 'hour']).agg(
    total_spend=('total', 'sum'),
    num_transactions=('datetime', 'count'),  # Use 'datetime' to count transactions
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Count items where trans_status is blank or a single space
).reset_index()


print(sales_by_hour)

      year  month  day  hour   total_spend  num_transactions  num_items
0     2010      1    1     9  0.000000e+00               254        175
1     2010      1    1    10  3.213000e+02              1019        771
2     2010      1    1    11 -2.258700e+02              1076        784
3     2010      1    1    12 -6.736000e+01              1109        779
4     2010      1    1    13  2.016000e+01              1562       1101
...    ...    ...  ...   ...           ...               ...        ...
1266  2010      3   31    18  9.394000e+01              2614       1745
1267  2010      3   31    19 -1.198500e+02              1921       1297
1268  2010      3   31    20  1.644400e+02              1770       1206
1269  2010      3   31    21 -1.644400e+02              1935       1380
1270  2010      3   31    22  2.664535e-15                26         14

[1271 rows x 7 columns]


2. Sales by owner by year by month:

In [20]:
# Create the 'Sales by owner by year by month' summary table
sales_by_owner = df.groupby(['card_no', 'year', 'month']).agg(
    total_sales=('total', 'sum'),
    num_transactions=('trans_status', 'count'),
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Correct logical OR operation
).reset_index()

print(sales_by_owner)

       card_no  year  month   total_sales  num_transactions  num_items
0      10001.0  2010      1  0.000000e+00               110         97
1      10001.0  2010      2  0.000000e+00                21         21
2      10001.0  2010      3  1.509903e-14                87         86
3      10003.0  2010      1  2.220446e-16                22         22
4      10003.0  2010      2  5.329071e-15                23         22
...        ...   ...    ...           ...               ...        ...
27087  51181.0  2010      2  0.000000e+00                 7          7
27088  51182.0  2010      1  3.552714e-15                52         35
27089  51182.0  2010      2  7.549517e-15                45         44
27090  51182.0  2010      3 -3.552714e-15                41         38
27091  51183.0  2010      3  8.881784e-16                 9          8

[27092 rows x 6 columns]


3. Sales by product description by year by month:

In [24]:
# Create the 'Sales by product description by year by month' summary table
sales_by_product = df.groupby(['upc', 'description', 'department', 'year', 'month']).agg(
    total_sales=('total', 'sum'),
    num_transactions=('trans_status', 'count'),
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Correct logical OR operation
).reset_index()

print(sales_by_product) ### NEED DEPARTMENT NAME FROM DEEPAMRTNETN KLOOK UP TABLE

            upc   description  department  year  month  total_sales  \
0             0  BREAD Coupon         7.0  2010      1        -3.00   
1             0  BREAD Coupon         7.0  2010      2        -2.00   
2             0  BREAD Coupon         7.0  2010      3        -1.00   
3             0   BULK Coupon         3.0  2010      1       -43.00   
4             0   BULK Coupon         3.0  2010      2       -84.00   
...         ...           ...         ...   ...    ...          ...   
49256  DISCOUNT      Discount         0.0  2010      2     -5702.97   
49257  DISCOUNT      Discount         0.0  2010      3     -8155.18   
49258       TAX           Tax         0.0  2010      1     32876.03   
49259       TAX           Tax         0.0  2010      2     32643.20   
49260       TAX           Tax         0.0  2010      3     34495.87   

       num_transactions  num_items  
0                     3          0  
1                     2          0  
2                     1          0  

Create SQLite Database and Insert Data

In [ ]:
# Create SQLite connection and cursor
conn = sqlite3.connect('wedge_coop_summary.db')
cursor = conn.cursor()

# Create the tables in SQLite
cursor.execute('''
CREATE TABLE sales_by_hour (
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER,
    total_spend REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_owner (
    card_no INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_product (
    upc TEXT,
    description TEXT,
    department INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

# Insert data into SQLite tables
# Insert sales by hour
sales_by_hour.to_sql('sales_by_hour', conn, if_exists='replace', index=False)

# Insert sales by owner
sales_by_owner.to_sql('sales_by_owner', conn, if_exists='replace', index=False)

# Insert sales by product
sales_by_product.to_sql('sales_by_product', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Summary tables created and stored in 'wedge_coop_summary.db'")
